In [ ]:
!pip install kafka-python

In [ ]:
!curl -sSOL https://dlcdn.apache.org/kafka/3.6.2/kafka_2.13-3.6.2.tgz
!tar -xzf kafka_2.13-3.6.2.tgz

In [ ]:
!./kafka_2.13-3.6.2/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.6.2/config/zookeeper.properties
!./kafka_2.13-3.6.2/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.6.2/config/server.properties
!echo "Esperamos 10 segundos hasta que los servicios kafka y zookeeper estén activos y funcionando"
!sleep 10

In [ ]:
!ps -ef | grep kafka

In [ ]:
!./kafka_2.13-3.6.2/bin/kafka-topics.sh --create --topic invernadero --bootstrap-server localhost:9092

In [ ]:
!./kafka_2.13-3.6.2/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092  --topic invernadero

In [ ]:
nombre_topico='invernadero'

In [ ]:
!cat producer_invernadero.py

In [ ]:
!nohup python -m producer_invernadero datos_invernaderi.csv invernadero 5 &

In [ ]:
!./kafka_2.13-3.6.2/bin/kafka-console-consumer.sh --topic invernadero --from-beginning --bootstrap-server localhost:9092

In [ ]:
from kafka import KafkaConsumer
import json


class Consumer:
    def __init__(self, topic):
        self._consumer = KafkaConsumer(topic,
                                       bootstrap_servers='localhost:9092',
                                       value_deserializer=lambda x: json.loads(x.decode('utf-8')),
                                       group_id='inver')

        self.data = []

    @property
    def consumer(self):
        return self._consumer

    @consumer.setter
    def consumer(self, value):
        if isinstance(value, KafkaConsumer):
            self._consumer = value

    def iniciar_consumicion(self):
        self.receive_message()

    def receive_message(self):
        message_count = 0
        for message in self._consumer:
            message = message.value
            print(f'Message {message_count}: {message}')
            self.data.append(message)
            message_count += 1

In [ ]:
consumer = Consumer(nombre_topico)
consumer.iniciar_consumicion()